In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [7]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
data_path = 'BinanceLOB/binance_dataset_2021-01-16_2194535408.csv'

In [9]:
lob_list = []

for side in ['bid','ask']:
    for i in range(100):
        lob_list.append(side + 'price' + str(i+1))
        lob_list.append(side + 'size' + str(i+1))
header_list = ['timestamp','last'];
header_list.extend(lob_list)

In [10]:
# Import data
binance_ethbtc = pd.read_csv(data_path, names=header_list, index_col='timestamp')

In [11]:
new_lob_order = []
for i in range(100):
    for side in ['bid','ask']:
        new_lob_order.append(side + 'price' + str(i+1))
        new_lob_order.append(side + 'size' + str(i+1))
new_lob_order = ['last'] + new_lob_order

In [12]:
binance_ethbtc = binance_ethbtc[new_lob_order]

In [13]:
def get_model_data(data, sample_size=600, feature_num=200):
    data = data.values
    shape = data.shape
    X = np.zeros((shape[0]-sample_size, sample_size, feature_num))
    Y = np.zeros(shape=(shape[0]-sample_size, 1))
    for i in range(shape[0]-sample_size):
        X[i] = data[i:i+sample_size,0:feature_num]# take the first feature_num columns as features
        Y[i] = data[i+sample_size-1,-1:]# take the last one column as labels
    X = X.reshape(X.shape[0], sample_size, feature_num, 1)# add the 4th dimension: 1 channel
    
    return X,Y

In [14]:
def define_y_labels(y, prediction_period, band_size = 0.001):
    bins = [-np.inf, -band_size, band_size, np.inf]
    names = [0, 1, 2]
    y_labels = pd.cut(y.pct_change(periods=prediction_period), bins, labels=names)
    return y_labels

In [15]:
datasample_period = 600
feature_columns = 40
prediction_period = 60

In [16]:
binance_ethbtc['delta_cat'] = define_y_labels(binance_ethbtc['last'], prediction_period)
weight_array = binance_ethbtc.groupby('delta_cat').count()['last']
weight_sum = weight_array.sum()

In [17]:
class_weight = {0: 1- weight_array[0]/weight_sum, 1: 1- weight_array[1]/weight_sum, 2: 1-weight_array[2]/weight_sum}

In [18]:
#split training and validation dataset
train_X, train_Y = get_model_data(binance_ethbtc.drop(['last'], axis=1), datasample_period, feature_columns)

In [19]:
train_y = to_categorical(train_Y[:])

In [20]:
# the size of a single input is (600,200)
input_tensor = Input(shape=(datasample_period,feature_columns,1))

# convolutional filter is (1,2) with stride of (1,2)
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,10))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

# Inception Module
tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

# concatenate features of tower_1, tower_2, tower_3
layer_x = layers.Reshape((600,96))(layer_x)

# 64 LSTM units
layer_x = LSTM(64)(layer_x)
# The last output layer uses a softmax activation function
output = layers.Dense(3, activation='softmax')(layer_x)
model = Model(input_tensor, output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 40, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 600, 20, 16)  48          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 600, 20, 16)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 600, 20, 16)  1040        leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [21]:
opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1)# learning rate and epsilon are the same as paper DeepLOB
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, batch_size=32, class_weight=class_weight)

Train on 6602 samples
Epoch 1/100
6602/6602 [==============================] - 34s 5ms/sample - loss: 0.5231 - accuracy: 0.6801
Epoch 2/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.4858 - accuracy: 0.6721
Epoch 3/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.4141 - accuracy: 0.6616
Epoch 4/100
6602/6602 [==============================] - 28s 4ms/sample - loss: 0.3598 - accuracy: 0.6990
Epoch 5/100
6602/6602 [==============================] - 28s 4ms/sample - loss: 0.3439 - accuracy: 0.7008
Epoch 6/100
6602/6602 [==============================] - 28s 4ms/sample - loss: 0.3095 - accuracy: 0.7384
Epoch 7/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.2813 - accuracy: 0.7663
Epoch 8/100
6602/6602 [==============================] - 28s 4ms/sample - loss: 0.2555 - accuracy: 0.7958
Epoch 9/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.2367 - accuracy: 0.8087
Epoch 10/100
6602/6602 [

6602/6602 [==============================] - 30s 5ms/sample - loss: 0.0701 - accuracy: 0.9423
Epoch 78/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0653 - accuracy: 0.9453
Epoch 79/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0599 - accuracy: 0.9518
Epoch 80/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0602 - accuracy: 0.9505
Epoch 81/100
6602/6602 [==============================] - 30s 5ms/sample - loss: 0.0589 - accuracy: 0.9538
Epoch 82/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0603 - accuracy: 0.9494
Epoch 83/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0580 - accuracy: 0.9520
Epoch 84/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0544 - accuracy: 0.9535
Epoch 85/100
6602/6602 [==============================] - 29s 4ms/sample - loss: 0.0575 - accuracy: 0.9543
Epoch 86/100
6602/6602 [==========================

In [24]:
model.save('my_model_ethbtc_v1.1_2021-01-16_2194535408')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model_ethbtc_v1.1_2021-01-16_2194535408\assets
